# ch07 输入/输出操作
本章介绍如下领域：

*基本I/O*
    
    **Python**有内建的函数，序列化对象、将其存储到磁盘、从磁盘将其读入RAM；除此之外，**Python**很擅长处理文本文件和**SQL**数据库。**Numpy**也提供了专用于快速存储和检索**ndarray**对象的函数。
 
**pandas** *的I/O*

    **pandas**库提供了丰富的便利函数及方法，读取不同格式存储的数据（例如csv、JSON），将数据写入不同格式的文件中。
    
**PyTables** *的I/O*

    **PyTables**使用HDF5标准实现大数据集的快速I/O操作；其速度往往只受到所使用硬件的限制。
## 7.1 Python基本I/O
### 7.1.1 将对象写入磁盘
使用**pickle**模块，可以序列化大部分**Python**对象，序列化指的是将对象（层次结构）转换为一个字节流；反序列化是相反的操作。下例中再次使用（伪）随机数据，这次保存在一个列表对象中：

In [1]:
path = 'D:/GitHub/python-study/python_for_finance/'

In [2]:
import numpy as np
from random import gauss
a = [gauss(1.5,2) for i in range(1000000)]

将列表对象写入磁盘供以后检索。**pickle**可以完成：

In [4]:
import pickle

In [5]:
pkl_file = open(path + 'data.pkl','w')

需要的两个重要函数是写入对象的**dump**和将对象加载到内存的**load**：

In [6]:
%time pickle.dump(a,pkl_file)

Wall time: 3.54 s


In [7]:
pkl_file

<open file 'D:/GitHub/python-study/python_for_finance/data.pkl', mode 'w' at 0x0000000009A2CC00>

In [8]:
pkl_file.close()

检查磁盘文件的大小，包含100万个浮点数的列表对象大约占据20兆字节（MB）磁盘空间：

In [9]:
#ll $path*

有了磁盘数据，就可以通过**pickle.load**将其读入内存：

In [10]:
pkl_file = open(path+'data.pkl','r')
%time b = pickle.load(pkl_file)

Wall time: 3.54 s


In [11]:
b[:5]

[-0.684280862083098,
 3.866452034479175,
 1.909527931440512,
 3.888071851306436,
 6.013572682098312]

将此与原始对象的前5个浮点对象比较：

In [12]:
a[:5]

[-0.684280862083098,
 3.866452034479175,
 1.909527931440512,
 3.888071851306436,
 6.013572682098312]

为了确保对象a和b相同，**Numpy**提供了**allclose**函数：

In [13]:
np.allclose(np.array(a),np.array(b))

True

原理上，这和计算两个**ndarray**对象的差，检查是否为0一样：

In [14]:
np.sum(np.array(a)-np.array(b))

0.0

然而，**allclose**有一个容错级别参数，默认设置为1e-5。

用**pickle**存储和读取单个对象明显相当简单，那么两个对象呢？

In [15]:
pkl_file = open(path + 'data.pkl','w')
%time pickle.dump(np.array(a),pkl_file)

Wall time: 1.28 s


In [16]:
%time pickle.dump(np.array(a)**2,pkl_file)

Wall time: 1.21 s


In [17]:
pkl_file.close()

上述操作：
1. 将原始对象的**ndarray**版本写入磁盘；
2. 还将**ndarray**的平方版本写入磁盘上的同一个文件；
3. 两个操作都比原来的快（因为使用**ndarray**对象）；
4. 文件的大小大致为以前的两倍；

下面将两个**ndarray**对象读回内存，显然，**pickle**按照先进先出（FIFO）原则保存对象：

In [20]:
pkl_file = open(path+'data.pkl','r')
x = pickle.load(pkl_file)
x

array([-0.68428086,  3.86645203,  1.90952793, ..., -1.115214  ,
        3.41076313,  1.55253036])

In [21]:
y = pickle.load(pkl_file)
y

array([  0.4682403 ,  14.94945133,   3.64629692, ...,   1.24370227,
        11.63330512,   2.41035053])

这种方法没有任何可用的元信息让用户事先知道保存在**pickle**文件中的是什么，可采用存储**字典（dict）**对象：

In [23]:
pkl_file = open(path+'data.pkl','w')
pickle.dump({'x':x,'y':y},pkl_file)
pkl_file.close()

此方法可以一次读取整组对象，还有一些其他好处，例如可以在字典对象的关键值上循环：

In [24]:
pkl_file = open(path+'data.pkl','r')
data = pickle.load(pkl_file)
pkl_file.close()
for key in data.keys():
    print key,data[key][:4]

y [  0.4682403   14.94945133   3.64629692  15.11710272]
x [-0.68428086  3.86645203  1.90952793  3.88807185]


不过这种方法要求我们一次写入和读取所有对象。
### 7.1.2 读写文本文件

In [25]:
rows = 5000
a = np.random.standard_normal((rows,5))
a.round(4)

array([[-0.3537,  0.285 ,  0.0062, -0.7421,  0.0045],
       [ 0.5187,  0.1505, -0.5658,  0.2235,  0.6948],
       [-0.2567, -0.4037, -0.4638,  0.0829, -0.9812],
       ..., 
       [-0.1889, -1.1675, -0.1379,  0.9567, -0.295 ],
       [ 0.3638, -0.4283, -0.8664,  0.7953,  0.9885],
       [ 0.8206, -0.7244, -1.206 , -0.858 ,  1.6389]])

In [26]:
import pandas as pd
t = pd.date_range(start='2014/1/1',periods=rows,freq='H')
t

DatetimeIndex(['2014-01-01 00:00:00', '2014-01-01 01:00:00',
               '2014-01-01 02:00:00', '2014-01-01 03:00:00',
               '2014-01-01 04:00:00', '2014-01-01 05:00:00',
               '2014-01-01 06:00:00', '2014-01-01 07:00:00',
               '2014-01-01 08:00:00', '2014-01-01 09:00:00',
               ...
               '2014-07-27 22:00:00', '2014-07-27 23:00:00',
               '2014-07-28 00:00:00', '2014-07-28 01:00:00',
               '2014-07-28 02:00:00', '2014-07-28 03:00:00',
               '2014-07-28 04:00:00', '2014-07-28 05:00:00',
               '2014-07-28 06:00:00', '2014-07-28 07:00:00'],
              dtype='datetime64[ns]', length=5000, freq='H')

In [38]:
csv_file = open(path+'data.csv','w')
header = 'date,no1,no2,no3,no4,no5\n'
csv_file.write(header)#写入数据列名称
for t_,(no1,no2,no3,no4,no5) in zip(t,a):
    s = '%s,%f,%f,%f,%f,%f\n' % (t_,no1,no2,no3,no4,no5)
    csv_file.write(s)
csv_file.close()

In [39]:
csv_file = open(path + 'data.csv','r')
for i in range(5):
    print csv_file.readline()

date,no1,no2,no3,no4,no5

2014-01-01 00:00:00,-0.353731,0.285017,0.006235,-0.742147,0.004483

2014-01-01 01:00:00,0.518702,0.150504,-0.565756,0.223530,0.694779

2014-01-01 02:00:00,-0.256681,-0.403742,-0.463805,0.082921,-0.981190

2014-01-01 03:00:00,0.669372,0.390002,0.020442,0.221858,1.088725



也可以使用**readlines**方法一次读入所有内容：

In [40]:
csv_file = open(path + 'data.csv','r')
content = csv_file.readlines()
for line in content[:5]:
    print line

date,no1,no2,no3,no4,no5

2014-01-01 00:00:00,-0.353731,0.285017,0.006235,-0.742147,0.004483

2014-01-01 01:00:00,0.518702,0.150504,-0.565756,0.223530,0.694779

2014-01-01 02:00:00,-0.256681,-0.403742,-0.463805,0.082921,-0.981190

2014-01-01 03:00:00,0.669372,0.390002,0.020442,0.221858,1.088725



In [41]:
content[:5]

['date,no1,no2,no3,no4,no5\n',
 '2014-01-01 00:00:00,-0.353731,0.285017,0.006235,-0.742147,0.004483\n',
 '2014-01-01 01:00:00,0.518702,0.150504,-0.565756,0.223530,0.694779\n',
 '2014-01-01 02:00:00,-0.256681,-0.403742,-0.463805,0.082921,-0.981190\n',
 '2014-01-01 03:00:00,0.669372,0.390002,0.020442,0.221858,1.088725\n']

In [42]:
csv_file.close()

### 7.1.3 SQL数据库
**Python**默认自带**SQLite3**数据库，利用这个数据库说明**Python**处理**SQL**数据库的方式：

In [43]:
import sqlite3 as sq3

**SQL**查询用字符串对象表示，语法、数据类型等取决于使用的数据库：

In [44]:
query = 'create table numbs(date date,no1 real,no2 real)'

打开一个数据库连接，本例中，在磁盘上生成一个新的数据库文件：

In [45]:
con = sq3.connect(path + 'numbs.db')

然后使用**execute**方法执行查询语句，创建一个表,并调用**commit**方法使查询生效：

In [47]:
con.execute(query)
con.commit()

可以在表中填入数据，每个数据行由日期-时间信息和两个浮点数组成：

In [50]:
import datetime as dt
#可以用单独的SQL语句写入单一数据行
con.execute('insert into numbs values(?,?,?)',
           (dt.datetime.now(),0.12,7.3))

In [53]:
a=con.execute('select * from numbs').fetchmany(10)
a

[(u'2017-12-10 15:53:09.393000', 0.12, 7.3)]

In [58]:
#批量写入较大的数据集
data = np.random.standard_normal((10000,2)).round(5)
for row in data:
    con.execute('insert into numbs values(?,?,?)',
                (dt.datetime.now(),row[0],row[1]))
con.commit()

In [72]:
#从数据库读取多行
con.execute('select * from numbs').fetchmany(10)

[(u'2017-12-10 15:53:09.393000', 0.12, 7.3),
 (u'2017-12-10 15:58:50', 0.7961, 1.58976),
 (u'2017-12-10 15:58:50.005000', 0.95556, -2.82312),
 (u'2017-12-10 15:58:50.005000', -1.7089, 0.41756),
 (u'2017-12-10 15:58:50.005000', 0.96038, -0.49005),
 (u'2017-12-10 15:58:50.005000', 0.29436, 1.73957),
 (u'2017-12-10 15:58:50.005000', -0.35771, 1.73986),
 (u'2017-12-10 15:58:50.005000', 0.93397, -1.46124),
 (u'2017-12-10 15:58:50.005000', 0.52877, 0.133),
 (u'2017-12-10 15:58:50.005000', 0.02756, 0.80837)]

In [73]:
#从数据库读取一行
pointer = con.execute('select * from numbs')
for i in range(3):
    print pointer.fetchone()

(u'2017-12-10 15:53:09.393000', 0.12, 7.3)
(u'2017-12-10 15:58:50', 0.7961, 1.58976)
(u'2017-12-10 15:58:50.005000', 0.95556, -2.82312)


In [74]:
con.close()

OperationalError: unable to close due to unfinalized statements or unfinished backups

### 7.1.4 读写NumPy数组
**NumPy**本身有以便利、高性能的方式写入和读取ndarray对象的函数。这在某些情况下节省了工作量，例如必须将**NumPy dtype**转换为特定数据库类型时。为了说明**NumPy**有时候时基于**SQL**方法的高效替代品，重复前面的例子，这次仅用**NumPy**：

In [75]:
import numpy as np

使用**NumPy**的**arange**函数替代**pandas**，生成保存日期时间（datetime）对象的数组对象：

In [77]:
dtimes = np.arange('2015-01-01 10:00:00','2021-12-31 22:00:00',dtype = 'datetime64[m]')
#minute intervals
len(dtimes)

3681360

**SQL**数据库中的表是**NumPy**中的结构数组，我们使用特殊的**dtype**对象镜像前面的**SQL**表：

In [79]:
dty = np.dtype([('Date','datetime64[m]'),('no1','f'),('no2','f')])
data = np.zeros(len(dtimes),dtype=dty)
data

array([('1970-01-01T00:00',  0.,  0.), ('1970-01-01T00:00',  0.,  0.),
       ('1970-01-01T00:00',  0.,  0.), ..., ('1970-01-01T00:00',  0.,  0.),
       ('1970-01-01T00:00',  0.,  0.), ('1970-01-01T00:00',  0.,  0.)],
      dtype=[('Date', '<M8[m]'), ('no1', '<f4'), ('no2', '<f4')])

用**日期（dates）**对象填充**Date**列：

In [80]:
data['Date'] = dtimes

其他两列和以前一样用伪随机数填充：

In [82]:
a = np.random.standard_normal((len(dtimes),2)).round(5)
data['no1']=a[:,0]
data['no2']=a[:,1]

**ndarray**对象的保存经过了高度优化，因此相当快速，在磁盘保存数据：

In [83]:
%time np.save(path+'array',data)

Wall time: 386 ms


读取数据更快：

In [84]:
%time np.load(path + 'array.npy')

Wall time: 35 ms


array([('2015-01-01T10:00',  1.36756003,  0.24412   ),
       ('2015-01-01T10:01',  0.53118998,  0.68303001),
       ('2015-01-01T10:02',  0.72872001,  0.43213999), ...,
       ('2021-12-31T21:57', -0.99317998, -1.26206994),
       ('2021-12-31T21:58', -1.53144002, -0.26495001),
       ('2021-12-31T21:59',  0.51980001,  1.54949999)],
      dtype=[('Date', '<M8[m]'), ('no1', '<f4'), ('no2', '<f4')])

在任何情况下都可预期，这种形式的数据存储和检索远快于**SQL**数据库或者使用标准**pickle**库的序列化。当然，使用这种方法当然没有**SQL**数据库的功能性，但是后面介绍的**PyTable**对此有所帮助。
## 7.2 Pandas的I/O
**pandas**库的主要优势之一是可以原生读取和写入不同数据格式，包括：CSV/SQL/XLS/XSLX/JSON/HTML等。

### 7.2.1 SQL数据库
### 7.2.2 从SQL到pandas
### 7.2.3 CSV文件数据
### 7.2.4 Excel文件数据
## 7.3 PyTable的快速I/O
### 7.3.1 使用表
### 7.3.2 使用压缩表
### 7.3.3 使用数组
### 7.3.4 内存外计算
## 7.4 结语
## 7.5 延伸阅读